In [ ]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import bw_processing as bwp
import numpy as np

if 'ei38-teaching-25' not in bd.projects:
    bi.restore_project_directory("/srv/data/projects/ecoinvent38-25.tar.gz")
    
bd.projects.set_current('ei38-teaching-25')

In [ ]:
bd.databases

Databases dictionary with 4 object(s):
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	biosphere3
	ei 3.8 cutoff

In [ ]:
db = bd.Database("ei 3.8 cutoff")

# Data building
The model will rely on datapackages for building the supply chain and experimenting

In [ ]:
#ecoinvent datapackage
data_objs_ei38 = db.datapackage()

In [ ]:
## Creating datapackage for each method
method_hint = "IPCC"
data_objs_methods = []
methods = [method for method in bd.methods if method_hint in method[0]]
for method in methods:
    _, objs, __ = bd.prepare_lca_inputs(method=method)
    data_objs_methods += objs

In [ ]:
_,method_test,__ = bd.prepare_lca_inputs(method=('IPCC 2013', 'climate change', 'GWP 100a'))


In [ ]:
## Identifying background processes in ecoinvent
db.search("wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future")

['wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future' (megajoule, CH, None),
 'wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future' (kilowatt hour, CH, None)]

In [ ]:
database_name = "ei 3.8 cutoff"
wood_pellet = bd.get_activity(
    database=database_name, 
    name='wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future', 
    unit = "megajoule"
    )

In [ ]:
wood_pellet.id

19364

In [14]:
my_node_id = 998877665544
my_node = np.array([
    [(my_node_id,my_node_id), 1, True], 
    [(wood_pellet.id,my_node_id), 1, False],    
              ])

indices = my_node[:,0].astype(bwp.INDICES_DTYPE)
data = my_node[:,1].astype(float)
flip = my_node[:,2].astype(bool)



/tmp/ipykernel_645196/1500133484.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  my_node = np.array([


In [104]:
## Creating the node representing "Operational Energy Moerschberg for 1 year"
dp = bwp.create_datapackage()

dp.add_persistent_vector(
    matrix="technosphere_matrix",
    data_array=data,
    indices_array=indices,
    flip_array=flip,
    name="example",
)

In [ ]:
lca = bc.LCA({my_node_id: 1}, data_objs=[data_objs_ei38] + data_objs_methods + [dp])
lca.lci()
lca.lcia()
lca.score

In [105]:
lca = bc.LCA({19364: 1}, data_objs=[data_objs_ei38] + method_test)
lca.lci()
lca.lcia()
lca.score

0.004655632602942411

0.00534001260793951